# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "data_file/cities.csv"
city_data = pd.read_csv(output_data_file)

city_data_df = pd.DataFrame(city_data)
city_data_df.sample(20)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
392,ponta delgada,75,PT,1609710340,87,37.73,-25.67,16.00,10.30
54,tual,90,ID,1609710314,76,-5.67,132.75,29.22,5.04
209,chirongui,20,YT,1609710325,83,-12.93,45.15,27.00,2.60
34,toba,75,JP,1609710312,60,34.48,136.85,6.00,7.70
459,saint-pierre,0,RE,1609710345,73,-21.34,55.48,24.00,3.10
107,hokitika,61,NZ,1609710319,77,-42.72,170.97,15.72,2.46
259,isangel,75,VU,1609709987,88,-19.55,169.27,28.00,2.60
427,loding,75,NO,1609710342,93,67.30,14.75,2.00,2.60
369,itoman,75,JP,1609710246,82,26.12,127.67,19.00,2.60
66,atuona,16,PF,1609709967,67,-9.80,-139.03,25.94,6.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_data_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_conditions_df = city_data_df

# A max temperature lower than 90 degrees but higher than 80(data was gathered in celsius).
# Wind speed less than 15 mph.
# Zero cloudiness.
ideal_conditions_df= ideal_conditions_df.loc[(ideal_conditions_df["Max Temp"] < 26) & (ideal_conditions_df["Max Temp"] > 21)]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Wind Speed"] < 15]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df["Cloudiness"] == 0]

# Drop any rows with null values
#ideal_conditions_df = ideal_conditions_df.dropna()

ideal_conditions_df.sample(20)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
269,cidreira,0,BR,1609709969,72,-30.18,-50.21,23.56,9.55
299,umm lajj,0,SA,1609710333,65,25.02,37.27,22.04,3.00
323,bilma,0,NE,1609709993,19,18.69,12.92,23.19,3.41
246,iquique,0,CL,1609710125,57,-20.22,-70.14,24.00,7.70
411,goundam,0,ML,1609709980,18,16.41,-3.67,25.09,4.58
75,pisco,0,PE,1609709968,78,-13.70,-76.22,23.00,10.80
500,kidal,0,ML,1609710348,21,18.44,1.41,23.31,3.47
511,arlit,0,NE,1609710348,20,18.74,7.39,23.67,1.83
454,nueva rosita,0,MX,1609710344,25,27.95,-101.22,21.67,2.24
521,grand gaube,0,MU,1609709998,85,-20.01,57.66,22.78,3.13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
